In [12]:
%matplotlib inline

In [13]:
import pandas as pd
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import glob,os

## Import fits image - big mosaic F139M

We need to import one of the WFC-IR mosaics (F139M in this case) to associate the sources detected multiple times.

In [14]:
#operator = 'mgabellini'
operator = 'robberto'

In [15]:
path_mos= '/Users/'+operator+'/Box Sync/Orion/lubeda_mosaics_03.29.2016/mosaic-f139m-flt-version12_drz_sci.fits'
hdu_mos = fits.open(path_mos)
hdu_mos.info()
image_mos = hdu_mos[0].data
header_mos = hdu_mos[0].header
filter_mos = header_mos['FILTER']
print(filter_mos)

Filename: /Users/robberto/Box Sync/Orion/lubeda_mosaics_03.29.2016/mosaic-f139m-flt-version12_drz_sci.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU    4315   (12400, 15000)   float32   
1    HDRTAB      BinTableHDU    557   208R x 274C   [9A, 3A, J, D, D, D, D, D, D, D, D, D, D, D, D, J, 2A, 9A, 7A, 18A, D, D, D, D, D, 3A, D, D, D, D, D, D, D, D, D, D, D, D, J, J, D, 3A, D, D, D, D, J, J, 8A, 23A, 11A, 18A, 4A, D, D, J, J, D, D, D, D, 23A, D, D, D, D, J, J, D, 3A, 8A, J, D, D, D, 23A, 1A, D, D, D, D, D, D, 12A, 12A, 8A, 23A, D, D, 10A, 10A, D, D, D, 2A, 3A, 3A, 4A, 8A, 7A, D, J, D, 6A, 9A, D, D, D, 9A, 18A, 3A, J, 5A, D, D, D, 8A, D, 3A, D, D, D, 3A, 1A, D, 23A, D, D, D, 3A, J, 1A, 4A, D, 3A, 6A, D, D, D, D, D, 23A, D, D, D, D, D, 1A, J, J, J, J, 8A, 23A, J, J, 10A, 7A, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, 13A, D, 24A, 23A, D, 1A, 1A, D, J, D, D, 1A, 1A, D, D, D, D, D, D, D, D, D, D, D, 4A, D, J, 23A, D, 8A, D, D, 13A, D, D, J,

Now we read the IR supercatalogue

In [16]:
# Do the same for the IR catalogue
rawpath_ir = "/Users/"+operator+"/Box Sync/Orion/photometry/Final_catalogue"
name_file_ir = rawpath_ir +"/Supertable.csv"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
data_ir = np.genfromtxt(name_file_ir,names=True,delimiter=',',\
                        dtype=[np.int,np.float64,np.float64,np.int,'S20','S20',np.int,np.int,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.int,np.float64,np.int,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.int,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.float64,np.int,np.int,np.int,np.int])


In [17]:
data_ir.dtype.names
#print(len(data_ir.dtype.names))

('index',
 'ra',
 'dec',
 'visit',
 'file_F130N',
 'file_F139M',
 'Extension',
 'Chip',
 'X',
 'Y',
 'Chi',
 'SNR',
 'Sharpness',
 'Roundness',
 'Axis',
 'Crowding',
 'Type',
 'Counts139',
 'Sky139',
 'Rate139',
 'Delta_Rate139',
 'VegaMag139',
 'UBVRI139',
 'DeltaMag139',
 'Chi139',
 'SNR139',
 'Sharp139',
 'Round139',
 'Crowd139',
 'flag139',
 'Counts130',
 'Sky130',
 'Rate130',
 'Delta_Rate130',
 'VegaMag130',
 'UBVRI130',
 'DeltaMag130',
 'Chi130',
 'SNR130',
 'Sharp130',
 'Round130',
 'Crowd130',
 'flag130',
 'fltID',
 'mainID',
 'FirstID')

In [18]:
print(data_ir['mainID'][20:40],data_ir['FirstID'][20:40])

(array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39]), array([20, 21, 22, 23, 24, 25, 26,  1,  5, 29, 30, 31, 32, 33, 34, 10, 36,
       16, 38, 39]))


In [19]:
d_ir = pd.DataFrame(data_ir)
pd.__version__
#d_ir.sort_values('FirstID',inplace=True)

u'0.18.0'

In [20]:
d_ir = pd.DataFrame(data_ir)
d_ir = d_ir.sort_values('FirstID')
uniq = pd.unique(d_ir['FirstID'])
N_uniche = len(uniq)
print(N_uniche, len(d_ir['FirstID']))

(5299, 8778)


In [21]:
#d_ir[['flag139','FirstID']]

In [22]:
ra_fin = np.ones(N_uniche, dtype=np.float)*(-99)
dec_fin = np.ones(N_uniche, dtype=np.float)*(-99)
mag130_fin = np.ones(N_uniche, dtype=np.float)*(-99)
err_mag130_fin = np.ones(N_uniche, dtype=np.float)*(-99)
delta130_fin= np.ones(N_uniche, dtype=np.float)*(-99)
flag130_fin = np.ones(N_uniche, dtype=np.float)*(-99)
mag139_fin = np.ones(N_uniche, dtype=np.float)*(-99)
err_mag139_fin = np.ones(N_uniche, dtype=np.float)*(-99)
delta139_fin= np.ones(N_uniche, dtype=np.float)*(-99)
flag139_fin = np.ones(N_uniche, dtype=np.float)*(-99)
N_found = np.ones(N_uniche, dtype=np.int)*(-99)
N_found_130 = np.ones(N_uniche, dtype=np.int)*(-99)
N_found_139 = np.ones(N_uniche, dtype=np.int)*(-99)
N_exp = np.ones(N_uniche, dtype=np.int)*(-99)
FirstID_fin = np.ones(N_uniche, dtype=np.int)*(-99)
print dec_fin

[-99. -99. -99. ..., -99. -99. -99.]


In [23]:
for k in range(0, N_uniche):
    #print(k)
    i_found = np.where(uniq[k] == data_ir['FirstID'])
    Nfound =len(i_found[0])
    #print(Nfound)
    i_found2 = np.where(data_ir[i_found]['flag130'] == 0)
    Nfound2 =len(i_found2[0])
    i_found3 = np.where(data_ir[i_found]['flag139'] == 0)
    Nfound3 =len(i_found3[0])
    if Nfound2 == 0:
        i_found2 = np.where(data_ir[i_found]['flag130'] == min(data_ir[i_found]['flag130']))
        Nfound2 =len(i_found2[0])
    if Nfound3 == 0:
        i_found3 = np.where(data_ir[i_found]['flag139'] == min(data_ir[i_found]['flag139']))
        Nfound3 =len(i_found3[0])
    #print(i_found[0][i_found3[0]])
    #print(Nfound2, Nfound3, Nfound)
    ra_fin[k] = np.mean(data_ir[i_found[0]]['ra'])
    dec_fin[k] = np.mean(data_ir[i_found[0]]['dec'])
    mag130_fin[k] = np.mean(data_ir[i_found[0][i_found2[0]]]['VegaMag130'])
    err_mag130_fin[k] = np.mean(data_ir[i_found[0][i_found2[0]]]['DeltaMag130'])
    delta130_fin[k] = (max(data_ir[i_found[0][i_found2[0]]]['VegaMag130'])-min(data_ir[i_found[0][i_found2[0]]]['VegaMag130']))
    flag130_fin[k] = np.mean(data_ir[i_found[0][i_found2[0]]]['flag130'])
    N_found_130[k] = Nfound2
    mag139_fin[k] = np.mean(data_ir[i_found[0][i_found3[0]]]['VegaMag139'])
    err_mag139_fin[k] = np.mean(data_ir[i_found[0][i_found3[0]]]['DeltaMag139'])
    delta139_fin[k] = (max(data_ir[i_found[0][i_found3[0]]]['VegaMag139'])-min(data_ir[i_found[0][i_found3[0]]]['VegaMag139']))
    flag139_fin[k] = np.mean(data_ir[i_found[0][i_found3[0]]]['flag139'])
    N_found_139[k] = Nfound3
    FirstID_fin[k] = data_ir[i_found[0][0]]['FirstID']
    N_found[k] = Nfound
    #N_exp[k] = N_expected

In [24]:
print data_ir['ra'][i_found][i_found2]

[ 83.94556217]


## ra-dec to x,y mosaic

In [25]:
a=ra_fin
b=dec_fin
world = np.array([a,b], np.float)
outra = world[0]
outdec = world[1]

In [26]:
from astropy import wcs
from astropy.wcs import WCS
#a=phot.X
#b=phot.Y
w= wcs.WCS(header_mos)
#w= wcs.WCS(hdu[0].header)
w.wcs.print_contents()
#pixcrd = np.array([a,b], np.float_)
#a2 = pixcrd[0]
#b2 = pixcrd[1]
n=len(a)
i = list(xrange(n))
pix1 = w.wcs_world2pix(outra, outdec, 1)
#world1 = w.wcs_pix2world(a2, b2, 1)
#print world1[0], world1[1]
outx = pix1[0]
outy = pix1[1]

In [27]:
slim_table = pd.DataFrame()
slim_table['ra'] = ra_fin
slim_table['dec'] = dec_fin
slim_table['X mos'] = outx
slim_table['Y mos'] = outy
slim_table['mag130'] = mag130_fin
slim_table['err_mag130'] = err_mag130_fin
slim_table['delta130'] = delta130_fin
slim_table['Flag 130'] = flag130_fin
slim_table['N 130'] = N_found_130
slim_table['mag139'] = mag139_fin
slim_table['err_mag139'] = err_mag139_fin
slim_table['delta139'] = delta139_fin
slim_table['Flag 139'] = flag139_fin
slim_table['N 139'] = N_found_139
slim_table['FirstID'] = FirstID_fin
slim_table['N'] = N_found
#slim_table[''] = N_exp

slim_table

,ra,dec,X mos,Y mos,mag130,err_mag130,delta130,Flag 130,N 130,mag139,err_mag139,delta139,Flag 139,N 139,FirstID,N
0,83.626381,-5.457797,10113.937075,7948.759999,10.180000,0.000000,0.317,0.0,3,9.470000,0.000000,0.018,12.0,2,0,3
1,83.609004,-5.441792,10599.642004,8397.894522,11.737500,0.001000,0.247,0.0,2,10.601000,0.000000,0.000,10.0,1,1,2
2,83.616348,-5.442953,10394.407530,8365.360953,12.153500,0.001000,0.263,0.0,2,11.312500,0.000000,0.243,10.0,2,2,2
3,83.632944,-5.443575,9930.648054,8348.017071,12.906500,0.001000,0.307,0.0,2,12.526000,0.000000,0.000,0.0,1,3,2
4,83.617540,-5.444363,10361.073057,8325.792596,13.940000,0.002000,0.462,0.0,2,13.302000,0.001000,0.000,2.0,1,4,2
5,83.600206,-5.440304,10845.504320,8439.596213,14.192667,0.002000,0.504,0.0,3,13.024000,0.001000,0.145,2.0,3,5,3
6,83.624975,-5.421476,10153.480378,8968.280101,13.757500,0.002000,0.429,0.0,2,13.440000,0.001000,0.006,0.0,2,6,2
7,83.622401,-5.431073,10225.339931,8698.870091,14.235500,0.002000,0.531,0.0,2,13.515000,0.001000,0.000,2.0,1,7,2
8,83.642761,-5.443064,9656.322571,8362.412655,16.878500,0.006000,0.011,0.0,2,16.144500,0.003000,0.181,0.0,2,8,2
9,83.619187,-5.461625,10314.943687,7841.241328,18.381333,0.017667,0.229,0.0,3,18.527000,0.008000,0.000,0.0,1,9,3


In [28]:
fileout = rawpath_ir + '/slim_table.csv'
slim_table.to_csv(fileout)

In [29]:
line1 = '# Region file format: DS9 version 4.1 \n'
line2 = '# Filename:',fileout, '\n'
line3 = 'global color=blue dashlist=8 3 width=1 font="helvetica 10 normal" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n'
line4 = 'physical \n'
n=len(outx)

#mxr-16-11-5: changed filename in 'slim_table.reg'
#with open(rawpath_ir + '/catalogue-ir_slim.reg', 'w') as f:
with open(rawpath_ir + '/slim_table.reg', 'w') as f:
    f.writelines(line1)
    f.writelines(line2)
    f.writelines(line3)
    f.writelines(line4)
    for i in range(n):
        f.writelines('point(%.4f %.4f) #circle=point \n'% (outx[i], outy[i]))



## We have to do the same on the F850LP catalogue

In [30]:
rawpath = "/Users/"+operator+"/Desktop/photometry/cross_corr_final"
name_file = rawpath+"/catalogue-f850lp.txt"                                                                                                                       
data = np.genfromtxt(name_file, names=True)#,dtype=[np.int,np.int,np.float64,np.float64,np.float64,np.float64,np.int])
#The original one
#rawpath = "/Users/"+operator+"/Desktop/photometry/cross_corr_final"
#name_file = glob.glob(rawpath+"/radec_Orion_850_total.txt")                                                                                                                          
#filer = np.genfromtxt(name_file,names=['ind', 'ra', 'dec', 'seq', 'ONCacs', 'n', 'a', 'b', 'c','d','e','f','f850lp','ef850lp','g','h'],dtype=[np.int,np.float64,np.float64,np.int,np.int,np.int,np.int,np.int,np.int,np.int,np.int,np.int,np.float64,np.float64,np.int,np.int])

IOError: /Users/robberto/Desktop/photometry/cross_corr_final/catalogue-f850lp.txt not found.

In [506]:
data['raf850lp']

array([ 83.541206,  83.542709,  83.546295, ...,  83.968071,  83.968636,
        83.971222])

In [507]:
data.dtype.names

('ind', 'unichek', 'raf850lp', 'dec850lp', 'f850lp', 'e850lp', 'N')

In [34]:
name_file_original = '/Users/'+operator+'/Box Sync/Orion/data catalogs/ACS/catalogue_2013.txt'                                                                                                                    
data_original = np.genfromtxt(name_file_original, names=True)
#data_original.dtype.names
print len(data_original['Seq'])

8185


In [35]:
d_original = pd.DataFrame(data_original)
#d_original

In [36]:
d_original = d_original.sort_values('ONCacs')
uniq_or = pd.unique(d_original['ONCacs'])
N_uniche_or = len(uniq_or)
print(N_uniche_or, len(d_original['ONCacs']))

(3399, 8185)


In [37]:
ra_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
dec_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
magF775W_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
err_magF775W_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
deltaF775W_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
flagF775W_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
magF850LP_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
err_magF850LP_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
deltaF850LP_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
flagF850LP_fin_or = np.ones(N_uniche_or, dtype=np.float)*(-99)
N_found_or = np.ones(N_uniche_or, dtype=np.int)*(-99)
type_or = np.ones(N_uniche_or, dtype=np.int)*(-99)
ONCacs_fin_or = np.ones(N_uniche_or, dtype=np.int)*(-99)
print dec_fin_or

[-99. -99. -99. ..., -99. -99. -99.]


In [38]:
for k in range(0, N_uniche_or):
    i_found_or = np.where(uniq_or[k] == data_original['ONCacs'])
    Nfound_or = len(i_found_or)
    #print(Nfound)
    ra_fin_or[k] = np.mean(data_original[i_found_or]['_RAJ2000deg'])
    dec_fin_or[k] = np.mean(data_original[i_found_or]['_DEJ2000deg'])
    magF775W_fin_or[k] = np.mean(data_original[i_found_or]['F775W'])
    err_magF775W_fin_or[k] = np.mean(data_original[i_found_or]['e_2'])
    deltaF775W_fin_or[k] = (max(data_original[i_found_or]['F775W'])-min(data_original[i_found_or]['F775W']))
    #flagF775W_fin_or[k] = np.mean(data_original[i_found_or]['f_3'])
    magF850LP_fin_or[k] = np.mean(data_original[i_found_or]['F850LP'])
    err_magF850LP_fin_or[k] = np.mean(data_original[i_found_or]['e_3'])
    #flagF850LP_fin_or[k] = np.mean(data_original[i_found_or]['f_4'])
    deltaF850LP_fin_or[k] = (max(data_original[i_found_or]['F850LP'])-min(data_original[i_found_or]['F850LP']))
    #type_or[k] = np.mean(data_original[i_found_or]['type'])
    ONCacs_fin_or[k] = data_original[i_found_or[0][0]]['ONCacs']
    N_found_or[k] = Nfound
    
print len(N_found_or)

3399


In [39]:
c=ra_fin_or
d=dec_fin_or
world_or = np.array([c,d], np.float)
outra_or = world_or[0]
outdec_or = world_or[1]

In [40]:
w= wcs.WCS(header_mos)
w.wcs.print_contents()
n_or=len(c)
i_or = list(xrange(n_or))
pix1_or = w.wcs_world2pix(outra_or, outdec_or, 1)
#world1 = w.wcs_pix2world(a2, b2, 1)
#print world1[0], world1[1]
outx_or = pix1_or[0]
outy_or = pix1_or[1]

In [42]:
slim_table_2013 = pd.DataFrame()
slim_table_2013['RA'] = ra_fin_or
slim_table_2013['DEC'] = dec_fin_or
slim_table_2013['X MOS'] = outx_or
slim_table_2013['Y MOS'] = outy_or
slim_table_2013['F775W'] = magF775W_fin_or
slim_table_2013['err_F775W'] = err_magF775W_fin_or
slim_table_2013['d_F775W'] = deltaF775W_fin_or
slim_table_2013['F850LP'] = magF850LP_fin_or
slim_table_2013['err_F850LP'] = err_magF850LP_fin_or
slim_table_2013['d_F850LP'] = deltaF850LP_fin_or
slim_table_2013['ONCacs'] = N_found_or
slim_table_2013

,RA,DEC,X MOS,Y MOS,F775W,err_F775W,d_F775W,F850LP,err_F850LP,d_F850LP,ONCacs
0,83.541204,-5.373961,12494.945711,10301.342218,21.400700,0.018200,0.0000,20.467300,0.01850,0.0000,1
1,83.542708,-5.364525,12453.011695,10566.230684,23.073800,0.101000,0.0000,21.739900,0.06580,0.0000,1
2,83.546296,-5.381794,12352.557694,10081.518126,13.115500,99.999900,0.0000,12.155500,99.99990,0.0000,1
3,83.546446,-5.344578,12348.748306,11126.190780,22.423400,0.033800,0.0000,21.419100,0.04190,0.0000,1
4,83.548421,-5.369708,12293.293731,10420.798973,21.979500,0.022500,0.0000,20.451100,0.01860,0.0000,1
5,83.548842,-5.335642,12281.874303,11377.052831,22.886000,0.033300,0.0000,21.189300,0.03510,0.0000,1
6,83.550067,-5.411064,12246.874043,9259.934476,23.226100,0.143200,0.0000,22.495800,0.06120,0.0000,1
7,83.550104,-5.405411,12245.897216,9418.616865,15.754100,0.001900,0.0000,14.702200,0.00210,0.0000,1
8,83.550262,-5.370828,12241.831190,10389.378299,21.557000,0.019200,0.0000,20.389600,0.02150,0.0000,1
9,83.550942,-5.419003,12222.341776,9037.092261,99.999900,24.937000,0.0000,21.723400,0.04280,0.0000,1


In [43]:
fileout_or = rawpath_ir + '/slim_table_2013.csv'
slim_table_2013.to_csv(fileout_or)

In [45]:
line1 = '# Region file format: DS9 version 4.1 \n'
line2 = '# Filename:',fileout_or, '\n'
line3 = 'global color=yellow dashlist=8 3 width=1 font="helvetica 10 normal" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n'
line4 = 'physical \n'
n_or=len(outx_or)
print n_or
#print(lines)

#
#mxr-161125: changed in slim_table_2013.reg 
#with open(rawpath_ir + '/catalogue-2013_slim.reg', 'w') as f:
with open(rawpath_ir + '/slim_table_2013.reg', 'w') as f:
    f.writelines(line1)
    f.writelines(line2)
    f.writelines(line3)
    f.writelines(line4)
    for i in range(n_or):
        f.writelines('point(%.4f %.4f) #circle=point \n'% (outx_or[i], outy_or[i]))



3399
